In [1]:
import pandas as pd

""" Testing sLM21 submission loading """
%env APP_DIR=/data/zerospeech/zrbench
from pathlib import Path

from zerospeech.benchmarks.sLM_21 import SLM21Submission

submission1 = SLM21Submission.load(Path('/data/zerospeech/zrbench/samples/sLM21-random-submission'))

env: APP_DIR=/data/zerospeech/zrbench


In [2]:
""" Testing abx-LS submission loading """
%env APP_DIR=/data/zerospeech/zrbench
from pathlib import Path

from zerospeech.benchmarks.abx_LS import AbxLSSubmission

submission2 = AbxLSSubmission.load(Path('/data/zerospeech/zrbench/samples/abxLS-random-submission/'))

env: APP_DIR=/data/zerospeech/zrbench


In [3]:
print(submission2.items.store.keys())

dict_keys(['dev_clean', 'test_clean', 'dev_other', 'test_other'])


In [9]:
""" Test validation of sLM21 """
%env APP_DIR=/data/zerospeech/zrbench
from zerospeech.benchmarks.sLM_21.validators import SLM21SubmissionValidator

validator = SLM21SubmissionValidator()

res = validator.validate(submission1)

env: APP_DIR=/data/zerospeech/zrbench


KeyboardInterrupt: 

In [ ]:
""" Test implementation of functions with internal state """
import random
from typing import List


def length_comparison(dim: int):
    data = dict(ncols=[], col_len=1)

    def comparison(array: List[str]):
        data['ncols'].append(len(array))
        res = []

        if len(set(data['ncols'])) != 1:
            print(f"found error {len(set(data['ncols']))}")
            res = ['error']
        return res

    return comparison


test_1 = [[random.randint(1, 30) for _ in range(100)] for _ in range(50)]
test_2 = [[random.randint(1, 30) for _ in range(100)] for _ in range(50)]

# add two at random
test_2[29].append(random.randint(1, 30))
test_2[19].append(random.randint(1, 30))

comp1 = length_comparison(2)
comp2 = length_comparison(2)

results1 = []
results2 = []
for i1, i2 in zip(test_1, test_2):
    results1.extend(comp1(i1))
    results2.extend(comp2(i2))

print(results1)
print(results2)

In [ ]:
""" Testing loading of scores dir and exporting into leaderboard """
%env APP_DIR=/data/zerospeech/zrbench
from zerospeech.benchmarks import sLM_21
from pathlib import Path
import pandas as pd
from rich.console import Console
from rich import pretty
pretty.install()


con = Console()

params = sLM_21.SLM21BenchmarkParameters()
dataset = sLM_21.SLM21Dataset.load()

dev_size = pd.read_csv(dataset.index.subsets.semantic_dev.items.pairs.file, header=0)\
    .groupby(['type', 'dataset'], as_index=False).size()
test_size = pd.read_csv(dataset.index.subsets.semantic_test.items.pairs.file, header=0)\
    .groupby(['type', 'dataset'], as_index=False).size()


scores = sLM_21.SLM21ScoreDir(
    location=Path('sLM-scores'),
    output_files=dict(
        lexical=params.lexical.result_filenames,
        syntactic=params.syntactic.result_filenames,
        semantic=params.semantic.result_filenames
    ),
    semantic_size=dict(dev=dev_size, test=test_size)
)

with con.status("Building leaderboard...", spinner="aesthetic"):
    led = scores.build_leaderboard()

with scores.leaderboard_file.open('w') as fp:
    fp.write(led.json(indent=4))

print("Leaderboard generated successfully")
# d = dict(led._iter(to_dict=True))